![https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3244747%2Fdf2bd9836c198215ed033d0678a02ef2%2Fheader.png?generation=1604778403014926&alt=media](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3244747%2Fdf2bd9836c198215ed033d0678a02ef2%2Fheader.png?generation=1604778403014926&alt=media)

In [ ]:
!pip install --upgrade plotly-geo
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime

## US Voting Results

In [ ]:
datafile = pd.read_csv('../input/us-election-2020/president_state.csv')
state_codes = pd.read_csv('../input/coordinates/world_country_and_usa_states_latitude_and_longitude_values.csv')
datafile = datafile.merge(state_codes, left_on='state', right_on='usa_state')

In [ ]:
fig = px.choropleth(datafile, locations='usa_state_code', color="total_votes",
                           range_color=(0, 10000000),
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 2020 USA Election: Vote Percentages by State

In [ ]:
#referennce : https://www.kaggle.com/paultimothymooney/2020-usa-election-vote-percentages-by-state

import warnings
warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM','REP'])]
df_president_county = df_president_county.groupby(['state','party'])["total_votes"].sum()
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/kaggle/input/coordinates/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes'] 
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes'] 
df_president_county = df_president_county.groupby(['state','usa_state_code'])["DEM_votes","REP_votes"].sum()
df_president_county = df_president_county.reset_index()
df_president_county['percent_democrat'] = df_president_county['DEM_votes']*100/(df_president_county['REP_votes']+df_president_county['DEM_votes'])

In [ ]:
fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',#blues
                    scope="usa",
                    title='2020 USA Election: Percent of Population Voting for the Democratic Party')
fig.show()

## Governor's County Results

In [ ]:
flips_main = pd.read_csv('../input/flips-for-maps/Flip_v2.csv')
flips = flips_main[['State','County Full Name','FIPS Full County']]
flips.columns = ['state','county','Flips']

In [ ]:
flips.head(2)

In [ ]:
flips.info()

In [ ]:
gov_county = pd.read_csv('../input/us-election-2020/governors_county.csv')
gov_county.head(2)

## Delaware

In [ ]:
Delaware_County = gov_county[gov_county['state'] == 'Delaware']
Delaware_Flips = flips[flips['state']=='Delaware']

In [ ]:
Delaware_County = Delaware_County.merge(Delaware_Flips, on='county', how='left')
Delaware_County = Delaware_County.drop(['state_y'],axis=1)
Delaware_County.head(2)

In [ ]:
county = Delaware_County['county'].tolist()
values = Delaware_County['current_votes'].tolist()
fips = Delaware_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Delaware'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Delaware'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Indiana

In [ ]:
Indiana_County = gov_county[gov_county['state'] == 'Indiana']
Indiana_Flips = flips[flips['state']=='Indiana']

In [ ]:
Indiana_Flips.info()

In [ ]:
Indiana_County = Indiana_County.merge(Indiana_Flips, on='county', how='left')
Indiana_County = Indiana_County.drop(['state_y'],axis=1)
Indiana_County.head(2)

In [ ]:
county = Indiana_County['county'].tolist()
values = Indiana_County['current_votes'].tolist()
fips = Indiana_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Indiana'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Indiana'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Missouri

In [ ]:
Missouri_County = gov_county[gov_county['state'] == 'Missouri']
Missouri_Flips = flips[flips['state']=='Missouri']

In [ ]:
Missouri_County = Missouri_County.merge(Missouri_Flips, on='county', how='left')
Missouri_County = Missouri_County.drop(['state_y'],axis=1)
Missouri_County.head(2)

In [ ]:
county = Missouri_County['county'].tolist()
values = Missouri_County['current_votes'].tolist()
fips = Missouri_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Missouri'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
    title='Missouri'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Montana

In [ ]:
Montana_County = gov_county[gov_county['state'] == 'Montana']
Montana_Flips = flips[flips['state']=='Montana']

In [ ]:
Montana_County = Montana_County.merge(Montana_Flips, on='county', how='left')
Montana_County = Montana_County.drop(['state_y'],axis=1)
Montana_County.head(2)

In [ ]:
county = Montana_County['county'].tolist()
values = Montana_County['current_votes'].tolist()
fips = Montana_County['Flips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Montana','North Dakota'], show_state_data=True,
    binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Total Votes by County',
    show_hover=True,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=False,
    title='Montana'
)
fig.update_layout(height=400, width=750, margin={"r":0,"t":0,"l":0,"b":0})
fig.layout.template = None
fig.show()

## Come back for more updates!!